In [1]:
import pandas as pd

In [2]:
data_path = './raw_datasets/movies.inter'

df = pd.read_csv(data_path,sep='\t')

In [3]:
df

,user_id:token,item_id:token,rating:float,timestamp:float
0,A3R5OBKS7OM2IR,0000143502,5.0,1358380800
1,A3R5OBKS7OM2IR,0000143529,5.0,1380672000
2,AH3QC2PC1VTGP,0000143561,2.0,1216252800
3,A3LKP6WPMP9UKX,0000143588,5.0,1236902400
4,AVIY68KEPQ5ZD,0000143588,5.0,1232236800
...,...,...,...,...
4607042,AHCV1RTGY3PJ8,B00LT1JHLW,5.0,1405641600
4607043,A2RWCXDMANY0LW,B00LT1JHLW,5.0,1405987200
4607044,A3V9PIFRME2XCW,B00LT1JHLW,5.0,1405900800
4607045,A3ROPC55BE2OM9,B00LT1JHLW,5.0,1405728000


In [4]:
df = df.rename(columns={"user_id:token": "session_id", "item_id:token": "item_id", "rating:float": "rating", "timestamp:float": "timestamp"})

In [ ]:
df.sort_values(by=['session_id','timestamp'], ignore_index=True)

In [5]:
df.groupby("session_id").count().describe()

,item_id,rating,timestamp
count,2.088620e+06,2.088620e+06,2.088620e+06
mean,2.205785e+00,2.205785e+00,2.205785e+00
std,1.136787e+01,1.136787e+01,1.136787e+01
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.000000e+00,1.000000e+00,1.000000e+00
50%,1.000000e+00,1.000000e+00,1.000000e+00
75%,2.000000e+00,2.000000e+00,2.000000e+00
max,2.654000e+03,2.654000e+03,2.654000e+03


In [6]:
df['tmp'] = 1.0

In [7]:
df.loc[df.rating < 4.0, 'tmp'] = 0.0

In [8]:
df = df.drop(columns='rating').rename(columns={'tmp': 'rating'})

In [9]:
len(df[df.rating == 1]), len(df)

(3618913, 4607047)

In [10]:
df.session_id.nunique(), df.item_id.nunique(), len(df)

(2088620, 200941, 4607047)

In [ ]:
df.describe()

In [ ]:
# df.to_csv("./Data/electronics_new.csv", index=False)

In [17]:
# sample
import numpy as np

np.random.seed(2022)

tot_item = df.item_id.nunique()

n_sampled_item = 100000

rand_id = df.item_id.unique()[np.random.choice(range(tot_item), n_sampled_item, replace=False)]

sampled_data = df[df.item_id.isin(rand_id)].reset_index(drop=True)

In [18]:
sampled_data.groupby("session_id").count().describe()

,item_id,timestamp,rating
count,1.260573e+06,1.260573e+06,1.260573e+06
mean,1.828219e+00,1.828219e+00,1.828219e+00
std,7.279172e+00,7.279172e+00,7.279172e+00
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.000000e+00,1.000000e+00,1.000000e+00
50%,1.000000e+00,1.000000e+00,1.000000e+00
75%,1.000000e+00,1.000000e+00,1.000000e+00
max,1.345000e+03,1.345000e+03,1.345000e+03


In [ ]:
# sampled_data.to_csv("Data/electronics_sampled.csv", index=False)

In [ ]:
# df.to_csv("Data/electronics.csv", index=False)

In [13]:
import logging
from pathlib import Path
import pickle
from os import path

import pandas as pd
from sklearn.utils import shuffle

from configuration.config import *

from dataloaders.utils import _drop_cold

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [ ]:
df

In [ ]:
sampled_data

In [19]:
hot = _drop_cold(sampled_data, 5, 5)

In [20]:
hot.session_id.nunique(), hot.item_id.nunique(), len(hot)

(47966, 21035, 625813)

In [ ]:
# hot.to_csv("Data/movies.csv", index=False)

In [16]:
hot.describe()

,session_id,item_id,timestamp,rating
count,218293.000000,218293.000000,2.182930e+05,218293.000000
mean,8493.242601,4181.684978,1.231644e+09,0.727847
std,4943.547377,2433.396359,1.300523e+08,0.445069
min,1.000000,1.000000,8.863776e+08,0.000000
25%,4257.000000,2097.000000,1.127606e+09,0.000000
50%,8447.000000,4129.000000,1.246925e+09,1.000000
75%,12749.000000,6284.000000,1.357690e+09,1.000000
max,17167.000000,8429.000000,1.406074e+09,1.000000


In [1]:
hot

NameError: name 'hot' is not defined